In [5]:
from dotenv import load_dotenv
import os
import json
import requests
import pandas as pd
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 \
import Features, EmotionOptions

In [6]:
load_dotenv()
print(os.environ.get('IBM_URL_SERVICE'))

https://api.au-syd.natural-language-understanding.watson.cloud.ibm.com/instances/9d6221a9-72c3-4785-97bb-8ebd7ff5a543


In [ ]:
df = pd.read_excel(r"C:\Users\Carlos\CAPSTONE-PROJECT\recursos\Tweets.xlsx");

In [9]:
print(df['text'])

0                  @VirginAmerica What @dhepburn said.
1    @VirginAmerica plus you've added commercials t...
2    @VirginAmerica I didn't today... Must mean I n...
3    @VirginAmerica it's really aggressive to blast...
4    @VirginAmerica and it's a really big bad thing...
5    @VirginAmerica seriously would pay $30 a fligh...
6    @VirginAmerica yes, nearly every time I fly VX...
7    @VirginAmerica Really missed a prime opportuni...
8    @virginamerica Well, I didn'tâ€¦but NOW I DO! :-D
Name: text, dtype: object


In [13]:
#def traducir_texto(text):
#EMPLEANDO NLP TRANSLATION API
#    url = "https://nlp-translation.p.rapidapi.com/v1/translate"

#    headers ={
#        "X-RapidAPI-Key": os.environ.get('NLP_API_KEY'),
#        "X-RapidAPI-Host": os.environ.get('NLP_API_HOST')
#    }
#    querystring = {"text":text,"to":"en"}
#    response = requests.get(url, headers=headers, params=querystring)

#    return response.json()

{'status': 200, 'from': 'ha', 'to': 'en', 'original_text': 'casa blanca', 'translated_text': {'en': 'White House'}, 'translated_characters': 11}


In [16]:
def promedio(matriz1, matriz2):
    #Obtener las dimensiones de ambas matrices
    fil = len(matriz1)
    col = len(matriz1[0])
    #Creamos la matriz promedio
    matriz_promedio = [[0] * col for _ in range(fil)]    
    #Multiplicación de matrices
    for i in range(fil):
        for j in range (col):
            promedio = (matriz1[i][j] + matriz2[i][j]) / 2
            matriz_promedio[i][j] = promedio
    return matriz_promedio

[[4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]
